In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：超参数调整表格回归模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_hyperparmeter_tuning_tabular_regression.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_hyperparmeter_tuning_tabular_regression.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库为自定义表格回归模型进行超参数调整。

数据集

本教程使用的数据集是[Boston房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。在本教程中使用的数据集版本已经内置到了TensorFlow中。训练好的模型可以预测以1千美元为单位的房屋的中位价格。

### 目标

在这个笔记本中，您将学习如何使用 Vertex 客户端库从 Python 脚本在 docker 容器中创建一个自定义表格回归模型的超参数调整作业。您也可以使用 `gcloud` 命令行工具或在 Google Cloud 控制台上在线进行超参数调整模型。


执行的步骤包括：

- 为训练自定义模型创建一个 Vertex 超参数调整作业。
- 调整自定义模型。
- 评估研究结果。

成本

本教程使用Google Cloud（GCP）的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的GA版本*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了顶点客户端库和Google *云存储*，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您有这个选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 Google 云项目

**无论您的笔记本环境如何，下面的步骤都是必需的。**

1. [选择或创建一个 Google 云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得 $ 300 的免费信用额度，用于您的计算/存储费用。

2. [确保项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 会将以 `!` 为前缀的行作为 shell 命令运行，并将以 `$` 为前缀的 Python 变量插入到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不会使用多区域存储桶用于Vertex的训练。并非所有区域都为所有Vertex服务提供支持。有关每个区域的最新支持信息，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能在使用共享测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并追加到在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云帐号

**如果您正在使用谷歌云笔记本**，您的环境已经经过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，并选择**Vertex 管理员**。在过滤框中输入“存储对象管理员”，并选择**存储对象管理员**。

点击创建。下载包含您密钥的 JSON 文件到您的本地环境。

将您的服务帐号密钥路径输入到下面的单元格中作为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您的笔记本环境如何，都需要按照以下步骤进行。**

当您使用Vertex客户端库提交自定义训练作业时，您需要将包含训练代码的Python包上传到云存储桶中。Vertex会从该包中运行代码。在本教程中，Vertex还会将作业结果生成的训练模型保存在同一个存储桶中。然后，您可以基于此输出创建一个`Endpoint`资源，以便提供在线预测。

请在下面设置您的云存储桶名称。存储桶名称必须在所有谷歌云项目中（包括您的组织之外的项目）是全局唯一的。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云储存存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点API服务端点。
- `PARENT`：用于数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

容器（Docker）映像

接下来，我们将设置用于训练的Docker容器映像。

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1`将是版本2.1，`1-15`将是版本1.15。以下列表显示了一些预构建的可用映像：

 - TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
 - TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
 - TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
 - TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
 - TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
 - Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### 机器类型

接下来，设置用于训练的机器类型。

- 将变量`TRAIN_COMPUTE`设定为配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU的内存为3.75GB。
     - `n1-highmem`：每个vCPU的内存为6.5GB。
     - `n1-highcpu`：每个vCPU的内存为0.9GB。
 - `vCPUs`：[\[2, 4, 8, 16, 32, 64, 96 \] ]个数

*注意：以下机器类型不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

# 教程

现在您已经准备好开始创建自己的超参数调整和训练自定义表格回归模型。

## 设置客户端

Vertex客户端库以客户端/服务器模式工作。在你的一边（Python脚本），你将创建一个客户端，用于向Vertex服务器发送请求和接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 用于`Model`资源的模型服务。
- 用于超参数调整的作业服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()

for client in clients.items():
    print(client)

调整模型 - 你好世界

有两种方法可以使用容器映像对模型进行超参数调整和训练：

- **使用Google Cloud预构建容器**。如果使用预构建容器，您将额外指定要安装到容器映像中的Python包。这个Python包包含了您用于超参数调整和训练自定义模型的代码。

- **使用您自己的自定义容器映像**。如果您使用自己的容器，容器需要包含您用于超参数调整和训练自定义模型的代码。

准备好超参数调整作业规范

现在客户准备就绪，你的第一步是为超参数调整作业创建作业规范。该作业规范将包括以下内容：

- `trial_job_spec`：自定义作业的规范。
  - `worker_pool_spec`：用于超参数调整的机器类型及数量的规范（单机或分布式）
  - `python_package_spec`：要与预构建容器一起安装的Python软件包的规范。

- `study_spec`：调整内容的规范。
  - `parameters`：这是你将为自定义训练作业调整的超参数的规范。它将包含一个列表。
  - `metrics`：这是每次调整试验结果如何评估的规范。

### 准备您的机器规格

现在为您的自定义超参数调整作业定义机器规格。这告诉 Vertex 需要为超参数调整提供何种类型的机器实例。
- `machine_type`：要提供的 GCP 实例的类型--例如，n1-standard-8。
- `accelerator_type`：加速器的类型，如果有的话。在本教程中，如果您之前设置了变量 `TRAIN_GPU != None`，则表示您正在使用 GPU；否则，您将使用 CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义超参数调整作业定义磁盘规格。这告诉Vertex应为超参数调整在每台机器实例中提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD或标准。SSD更快，标准更便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘大小，单位为GB。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

定义工作池规范

接下来，您需要定义自定义超参数调整作业的工作池规范。工作池规范将包括以下内容：

- `replica_count`：要提供的此类型机器的实例数。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要在VM实例上安装的Python训练包，以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们更深入地了解Python包规范：

- `executor_image_spec`：这是为您的自定义超参数调整作业配置的Docker镜像。

- `package_uris`：这是要在提供的实例上安装的Python训练包的位置（URI）列表。这些位置需要位于Cloud Storage存储桶中。这些可以是单独的python文件，也可以是整个包的zip（归档）。在后一种情况下，作业服务将解压缩（解压缩）内容到docker镜像中。

- `python_module`：要调用以运行自定义超参数调整作业的Python模块（脚本）。在此示例中，您将调用`trainer.task.py` -- 请注意，这里不需要附加`.py`后缀。

- `args`：要传递给相应Python模块的命令行参数。在此示例中，您将设置：
  - `"--model-dir=" + MODEL_DIR`：存储模型工件的Cloud Storage位置。有两种方法可以告诉超参数调整脚本在哪里保存模型工件：
      - 直接：将Cloud Storage位置作为命令行参数传递给训练脚本（设置变量`DIRECT = True`），
      - 间接：作业服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的时代数。
  - `"--steps=" + STEPS`：每个时代的步数（批次数）。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单一或分布式超参数调整的超参数调整分布策略。
     - `"single"`：单个设备。
     - `"mirror"`：单个计算实例上的所有GPU设备。
     - `"multi"`：所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_boston.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 创建一个研究规范

让我们从一个简单的研究开始。您只需要使用一个参数——*学习率*。由于只有一个参数，随机搜索并没有太多意义。相反，我们将对一系列数值进行网格搜索。

- `度量标准`：
 - `度量标准id`：在这个示例中，要报告的客观指标是 `'val_accuracy'`。
 - `目标`：在这个示例中，超参数调整服务将评估试验以最大化客观指标的价值。
- `参数`：调整的超参数规范。
 - `参数id`：将被传递给Python包作为命令行参数的超参数的名称。
  - `缩放类型`：缩放类型决定当搜索搜索空间时超参数调整服务使用的分辨率。
   - `单位线性缩放`：在搜索空间中的任何地方使用相同的分辨率。
   - `单位对数缩放`：靠近搜索空间底部的值距离更远。
   - `单位反对数缩放`：靠近搜索空间顶部的值距离更远。
 - **搜索空间**：在这里，您将指定要选择进行调整的超参数值的搜索空间。
   - `整数值规范`：指定在 `min_value` 和 `max_value` 之间的整数范围的值。
   - `双精度值规范`：指定在 `min_value` 和 `max_value` 之间的连续值范围。
   - `离散值规范`：指定一个值列表。
- `算法`：选择每次试验的超参数值的搜索方法：
 - `网格搜索`：组合搜索--在这个示例中使用。
 - `随机搜索`：随机搜索。

In [ ]:
study_spec = {
    "metrics": [
        {
            "metric_id": "val_accuracy",
            "goal": aip.StudySpec.MetricSpec.GoalType.MAXIMIZE,
        }
    ],
    "parameters": [
        {
            "parameter_id": "lr",
            "discrete_value_spec": {"values": [0.001, 0.01, 0.1]},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        }
    ],
    "algorithm": aip.StudySpec.Algorithm.GRID_SEARCH,
}

### 组装超参数调整工作规范

现在组装自定义超参数调整规范的完整描述：

- `display_name`：您为此自定义超参数调整工作分配的人类可读名称。
- `trial_job_spec`：自定义超参数调整工作的规范。
- `study_spec`：要调整的规范。
- `max_trial_count`：调整试验的最大数量。
- `parallel_trial_count`：要并行尝试的试验数量；否则，它们是顺序进行的。

In [ ]:
hpt_job = {
    "display_name": JOB_NAME,
    "trial_job_spec": {"worker_pool_specs": worker_pool_spec},
    "study_spec": study_spec,
    "max_trial_count": 6,
    "parallel_trial_count": 1,
}

### 检查超参数调整包

#### 包布局

在开始超参数调整之前，您将查看一个用于自定义超参数调整工作的Python包是如何组装的。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 镜像的操作环境中的说明。

文件 `trainer/task.py` 是用于执行自定义超参数调整工作的Python脚本。*注意*，当我们在工作池规范中引用它时，我们将目录斜杠替换为点(`trainer.task`)，并丢弃文件后缀(`.py`)。

#### 包装配

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python hyperparameter tuning script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration hyperparameter tuning script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py内容

在下一个单元格中，您将编写超参数调整脚本task.py的内容。我不会详细说明，只是供您查看。总结如下：

- 将试验的超参数值作为命令行参数传递（`parser.add_argument('--lr',...)`）
- 模拟一个训练循环，在每次循环（周期）中，将变量`accuracy`设置为循环迭代*学习率。
- 使用`report_hyperparameter_tuning_metric()`将客观指标`accuracy`报告回超参数调整服务。

In [ ]:
%%writefile custom/trainer/task.py
# HP Tuning hello world example

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import argparse
import os
import sys
import time
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--model-dir',
                    dest='model_dir',
                    default='/tmp/saved_model',
                    type=str,
                    help='Model dir.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print(device_lib.list_local_devices())

# Instantiate the HyperTune reporting object
hpt = HyperTune()

for epoch in range(1, args.epochs+1):
    # mimic metric result at the end of an epoch
    acc = args.lr * epoch
    # save the metric result to communicate back to the HPT service
    hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_accuracy',
        metric_value=acc,
        global_step=epoch)
    print('epoch: {}, accuracy: {}'.format(epoch, acc))
    time.sleep(1)

将超参数调整脚本存储在您的云存储桶中

接下来，将超参数调整文件夹打包成一个压缩的tar球，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_boston.tar.gz

### 使用 hypertune 报告试验结果

对于每次试验，您的Python脚本需要向超参数调整服务报告您指定为评估试验的目标指标。

在本示例中，您将在研究规范中指定目标指标将作为 `val_loss` 报告回来。

您可以使用 `HyperTune` 报告目标指标的值。该Python模块用于将关键/值对传达到超参数调整服务。要设置Python包中的报告，您需要添加以下三个步骤的代码：

1. 导入HyperTune模块：`from hypertune import HyperTune()`.
2. 在每个epoch结束时，使用 `hpt.report_hyperparameter_tuning_metric()` 将目标函数的当前值作为关键/值对写入日志。在本例中，参数为：
- `hyperparameter_metric_tag`: 要报告回去的目标指标的名称。名称必须与研究规范中指定的名称完全相同。
- `metric_value`: 要报告回超参数服务的目标指标的值。
- `global_step`: epoch迭代，从0开始。

## 超参数调优模型

现在在Vertex上开始调优您的自定义模型的超参数调优。使用这个辅助函数 `create_hyperparameter_tuning_job`，它接受以下参数：

-`hpt_job`: 超参数调优作业的规范。

这个辅助函数调用作业客户端服务的 `create_hyperparameter_tuning_job` 方法，带有以下参数：

-`parent`: 指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
-`hyperparameter_tuning_job`: 超参数调优作业的规范。

您将显示一些在`response`对象中返回的字段，其中最感兴趣的两个是：

`response.name`: 为此自定义超参数调优作业分配的Vertex完全合格的标识符。您将保存这个标识符以在后续步骤中使用。

`response.state`: 自定义超参数调优作业的当前状态。

In [ ]:
def create_hyperparameter_tuning_job(hpt_job):
    response = clients["job"].create_hyperparameter_tuning_job(
        parent=PARENT, hyperparameter_tuning_job=hpt_job
    )
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_hyperparameter_tuning_job(hpt_job)

现在获取您创建的超参数调整作业的唯一标识符。

In [ ]:
# The full unique ID for the hyperparameter tuning job
hpt_job_id = response.name
# The short numeric ID for the hyperparameter tuning job
hpt_job_short_id = hpt_job_id.split("/")[-1]

print(hpt_job_id)

### 获取超参数调整作业的信息

接下来，使用这个辅助函数 `get_hyperparameter_tuning_job`，它接受以下参数：

- `name`: 超参数调整作业的Vertex完全限定标识符。

该辅助函数调用作业客户服务的 `get_hyperparameter_tuning_job` 方法，并传入以下参数：

- `name`: 超参数调整作业的Vertex完全限定标识符。

如果你记得，在调用 `create_hyperparameter_tuning_job` 方法时，你通过 `response.name` 字段获取了超参数调整作业的Vertex完全限定标识符，并把这个标识符保存在变量 `hpt_job_id` 中。

In [ ]:
def get_hyperparameter_tuning_job(name, silent=False):
    response = clients["job"].get_hyperparameter_tuning_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_hyperparameter_tuning_job(hpt_job_id)

## 请等待调整完成

调整上述模型的超参数可能需要20分钟以上的时间。

一旦您的模型调整完毕，您可以通过将`end_time`减去`start_time`来计算调整模型所需的实际时间。

对于您的模型，我们需要知道每次试验保存模型的位置，在Python脚本中保存在您的本地云存储桶中的`MODEL_DIR + '/<trial_number>/saved_model.pb'`。

In [ ]:
while True:
    job_response = get_hyperparameter_tuning_job(hpt_job_id, True)
    if job_response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Study trials have not completed:", job_response.state)
        if job_response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        print("Study trials have completed")
        break
    time.sleep(60)

###审查研究结果

现在审查试验结果。

In [ ]:
best = (None, None, None, 0.0)
for trial in job_response.trials:
    print(trial)
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

现在看看哪个试验是最好的：### 最佳试验

In [ ]:
print("ID", best[0])
print("Learning Rate", best[1])
print("Decay", best[2])
print("Validation Accuracy", best[3])

## 获得最佳模型

如果您使用了让服务告诉调优脚本在哪里保存模型工件的方法（`DIRECT = False`），那么最佳模型的模型工件将保存在：

    MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

调整模型 - 波士顿房屋

现在你已经看到了使用模拟训练模型的Python包来调整超参数的自定义训练作业的总体步骤，你将为波士顿房屋模型进行一个新的超参数调整作业。

在这个例子中，你将改变两个部分：

1. 指定波士顿房屋自定义超参数调整的Python包。
2. 指定一个针对波士顿房屋自定义超参数调整Python包中使用的超参数的研究规范。

### 创建研究规范

在这项研究中，您将使用随机搜索算法调整两个超参数：

- **学习率**：搜索空间是一组离散值。
- **学习率衰减**：搜索空间是在1e-6和1e-2之间的连续范围。

目标是最大化验证精度。

您将运行最多六次试验。

In [ ]:
study_spec = {
    "metrics": [
        {"metric_id": "val_loss", "goal": aip.StudySpec.MetricSpec.GoalType.MAXIMIZE}
    ],
    "parameters": [
        {
            "parameter_id": "lr",
            "discrete_value_spec": {"values": [0.001, 0.01, 0.1]},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        },
        {
            "parameter_id": "decay",
            "double_value_spec": {"min_value": 1e-6, "max_value": 1e-2},
            "scale_type": aip.StudySpec.ParameterSpec.ScaleType.UNIT_LINEAR_SCALE,
        },
    ],
    "algorithm": aip.StudySpec.Algorithm.RANDOM_SEARCH,
}

### 组装超参数调整作业规范

现在组装自定义超参数调整规范的完整描述:

- `display_name`:您分配给此自定义超参数调整作业的易读名称。
- `trial_job_spec`: 自定义超参数调整作业的规范。
- `study_spec`: 要调整的内容规范。
- `max_trial_count`: 最大调整试验次数。
- `parallel_trial_count`: 并行尝试的试验次数；否则，它们将按顺序执行。

In [ ]:
hpt_job = {
    "display_name": JOB_NAME,
    "trial_job_spec": {"worker_pool_specs": worker_pool_spec},
    "study_spec": study_spec,
    "max_trial_count": 6,
    "parallel_trial_count": 1,
}

#### Task.py 的内容

在下一个单元格中，您可以看到超参数调优脚本 task.py 的内容。我不会详细展开，只是让您浏览一下。总结如下：

- 解析当前试验的超参数设置的命令行参数。
- 从命令行中获取保存模型文件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 下载并预处理波士顿住房数据集。
- 构建一个深度神经网络模型。
- 在构建和编译模型时使用每个密集层的单元数量，以及学习率超参数值。
- 定义一个回调 `HPTCallback`，在每个周期结束时获取验证损失，并通过 `hpt.report_hyperparameter_tuning_metric()` 报告给超参数调优服务。
- 使用 `fit()` 方法训练模型，并指定一个回调，将验证损失报告给超参数调优服务。

In [ ]:
%%writefile custom/trainer/task.py
# Custom Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--decay', dest='decay',
                    default=0.98, type=float,
                    help='Decay rate')
parser.add_argument('--units', dest='units',
                    default=64, type=int,
                    help='Number of units.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)

# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(args.units, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(args.units, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr, decay=args.decay))
  return model


model = build_and_compile_dnn_model()

# Instantiate the HyperTune reporting object
hpt = HyperTune()

# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_loss',
        metric_value=logs['val_loss'],
        global_step=epoch)

# Train the model
BATCH_SIZE = 16
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=BATCH_SIZE, validation_split=0.1, callbacks=[HPTCallback()])
model.save(args.model_dir)

####将超参数调整脚本存储在您的云存储桶中

接下来，将超参数调整文件夹打包成压缩的tar文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_boston.tar.gz

使用 hypertune 报告试验结果

对于每个试验，您的 Python 脚本需要向超参数调整服务报告您指定为评估试验的标准的客观指标。

在本例中，您将在研究规范中指定客观指标将被报告为 `val_loss`。

您将使用 `HyperTune` 报告客观指标的值。这个 Python 模块用于与超参数调整服务通信。要在您的 Python 包中设置这个报告，您需要添加以下三个步骤的代码：

1. 导入 HyperTune 模块：`from hypertune import HyperTune()`.
2. 在每个 epoch 结束时，使用 `hpt.report_hyperparameter_tuning_metric()` 将客观函数的当前值写入日志作为键值对。在本例中，参数为：
 - `hyperparameter_metric_tag`：要报告的客观指标的名称。名称必须与研究规范中指定的名称相同。
 - `metric_value`：要报告给超参数服务的客观指标的值。
 - `global_step`：epoch 迭代，从 0 开始。

## 超参数调整模型

现在开始在Vertex上对您的自定义模型进行超参数调整。使用此辅助函数 `create_hyperparameter_tuning_job`，它接受以下参数：

-`hpt_job`：超参数调整作业的规范。

辅助函数调用作业客户端服务的 `create_hyperparameter_tuning_job` 方法，具有以下参数：

-`parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
-`hyperparameter_tuning_job`：超参数调整作业的规范。

您将显示`response`对象中返回的一些字段，其中最感兴趣的两个字段是：

`response.name`：为此自定义超参数调整作业分配的Vertex完全合格标识符。您保存此标识符以在后续步骤中使用。

`response.state`：自定义超参数调整作业的当前状态。

In [ ]:
def create_hyperparameter_tuning_job(hpt_job):
    response = clients["job"].create_hyperparameter_tuning_job(
        parent=PARENT, hyperparameter_tuning_job=hpt_job
    )
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_hyperparameter_tuning_job(hpt_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
hpt_job_id = response.name
# The short numeric ID for the custom job
hpt_job_short_id = hpt_job_id.split("/")[-1]

print(hpt_job_id)

### 获取超参数调整作业的信息

接下来，使用这个辅助函数 `get_hyperparameter_tuning_job`，它接受以下参数：

- `name`：超参数调整作业的 Vertex 完全限定标识符。

这个辅助函数调用作业客户服务的 `get_hyperparameter_tuning_job` 方法，参数为：

- `name`：超参数调整作业的 Vertex 完全限定标识符。

如果你记得，当你调用 `create_hyperparameter_tuning_job` 方法时，在 `response.name` 字段中你获取了超参数调整作业的 Vertex 完全限定标识符，然后将该标识符保存在变量 `hpt_job_id` 中。

In [ ]:
def get_hyperparameter_tuning_job(name, silent=False):
    response = clients["job"].get_hyperparameter_tuning_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_hyperparameter_tuning_job(hpt_job_id)

等待调整完成

对上述模型进行超参数调整可能需要20分钟以上的时间。

一旦您的模型调整完成，您可以通过将`end_time`减去`start_time`来计算调整模型所需的实际时间。

对于您的模型，我们需要知道每次试验保存模型的位置，在Python脚本中保存在您的本地云存储桶中的`MODEL_DIR + '/<trial_number>/saved_model.pb'`。

In [ ]:
while True:
    job_response = get_hyperparameter_tuning_job(hpt_job_id, True)
    if job_response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Study trials have not completed:", job_response.state)
        if job_response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        print("Study trials have completed")
        break
    time.sleep(60)

### 请审查研究的结果

现在审查试验的结果。

In [ ]:
best = (None, None, None, 0.0)
for trial in job_response.trials:
    print(trial)
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

现在看看哪个试验是最好的：

In [ ]:
print("ID", best[0])
print("Learning Rate", best[1])
print("Decay", best[2])
print("Validation Accuracy", best[3])

获取最佳模型

如果您使用了让服务告诉调优脚本在哪里保存模型工件的方法（‘DIRECT = False’），那么最佳模型的模型工件将保存在：

MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶加载它，然后您可以做一些事情，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，传递保存模型的 Cloud Storage 路径，由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们找出模型的表现有多好。

### 加载评估数据

您将从 `tf.keras.datasets` 加载波士顿房屋测试（留存）数据，使用 `load_data()` 方法。这将返回数据集作为两个元素的元组。第一个元素是训练数据，第二个是测试数据。每个元素也是两个元素的元组：特征数据和对应的标签（自住房屋的中位价）。

您不需要训练数据，这就是为什么我们将其加载为 `(_, _)`。

在您能够通过评估数据之前，您需要对其进行预处理：

x_test：
1. 标准化（重新缩放）每一列中的数据，方法是将每个值除以该列的最大值。这将用 0 到 1 之间的 32 位浮点数替换每个单个值。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

执行模型评估

现在评估一下定制作业中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

清理工作

要清理本项目中使用的所有 GCP 资源，您可以删除用于本教程的 GCP 项目。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME